# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
pip install cartopy geoviews pyproj


     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ---------------------------------------- 56.0/56.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of panel to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 15.9 MB/s eta 0:00:01
   ---- ----------------------------------- 1.3/11.0 MB 16.8 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/11.0 MB 17.6 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/11.0 MB 17.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.7/11.0 MB 17.0 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 16.9 MB/s eta 0:00:01
   ------------------- -------------------- 5.3/11.0 MB 16.8 MB/s eta 0:00:01
   --------------------- ------------------ 5.9/11.0 MB 16.4 MB/s eta 0:00:01
   --------

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,12.07,72,0,5.58,AU,1741119689
1,1,adamstown,-25.0660,-130.1015,25.56,77,14,6.41,PN,1741119690
2,2,west island,-12.1568,96.8225,27.99,89,40,4.63,CC,1741119692
3,3,port-aux-francais,-49.3500,70.2167,6.00,76,81,8.22,TF,1741119693
4,4,bethel,41.3712,-73.4140,9.13,64,100,4.12,US,1741119695


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,72,mar del plata,-38.0023,-57.5575,23.86,85,0,2.57,AR,1741119650
200,200,malvan,16.0667,73.4667,26.59,52,0,2.38,IN,1741119935
325,325,alice springs,-23.7000,133.8833,21.75,28,0,0.00,AU,1741120091
327,327,turan,16.0678,108.2208,24.99,94,0,0.51,VN,1741120093
358,358,mount isa,-20.7333,139.5000,21.87,60,0,1.54,AU,1741120129
378,378,mukalla,14.5425,49.1242,23.04,71,0,3.25,YE,1741120154
383,383,broken hill,-31.9500,141.4333,23.56,35,0,4.15,AU,1741120161
460,460,ajman city,25.4111,55.4350,23.02,60,0,3.60,AE,1741120253
509,509,charters towers,-20.1000,146.2667,22.40,92,0,3.33,AU,1741120318
513,513,buckeye,33.3703,-112.5838,21.49,25,0,2.06,US,1741120323


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
72,mar del plata,AR,-38.0023,-57.5575,85,
200,malvan,IN,16.0667,73.4667,52,
325,alice springs,AU,-23.7000,133.8833,28,
327,turan,VN,16.0678,108.2208,94,
358,mount isa,AU,-20.7333,139.5000,60,
378,mukalla,YE,14.5425,49.1242,71,
383,broken hill,AU,-31.9500,141.4333,35,
460,ajman city,AE,25.4111,55.4350,60,
509,charters towers,AU,-20.1000,146.2667,92,
513,buckeye,US,33.3703,-112.5838,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mar del plata - nearest hotel: Hotel 3 de Abril
malvan - nearest hotel: Hotel Silver Sand
alice springs - nearest hotel: No hotel found
turan - nearest hotel: Thanh Thanh Hotel
mount isa - nearest hotel: Ibis Styles
mukalla - nearest hotel: فندق الجابري
broken hill - nearest hotel: Ibis Styles
ajman city - nearest hotel: Safari Hotel Apartments
charters towers - nearest hotel: No hotel found
buckeye - nearest hotel: Days Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
72,mar del plata,AR,-38.0023,-57.5575,85,Hotel 3 de Abril
200,malvan,IN,16.0667,73.4667,52,Hotel Silver Sand
325,alice springs,AU,-23.7000,133.8833,28,No hotel found
327,turan,VN,16.0678,108.2208,94,Thanh Thanh Hotel
358,mount isa,AU,-20.7333,139.5000,60,Ibis Styles
378,mukalla,YE,14.5425,49.1242,71,فندق الجابري
383,broken hill,AU,-31.9500,141.4333,35,Ibis Styles
460,ajman city,AE,25.4111,55.4350,60,Safari Hotel Apartments
509,charters towers,AU,-20.1000,146.2667,92,No hotel found
513,buckeye,US,33.3703,-112.5838,25,Days Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
 %%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)